# Classical Time Series Forecasting Methods

This notebook demonstrates classical forecasting methods including ARIMA and Prophet.

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.models.arima_model import ARIMAForecaster
from src.models.prophet_model import ProphetForecaster
from src.utils.helpers import generate_sample_data
from src.preprocessing.data_loader import train_test_split_ts, create_dataset_for_prophet
from src.evaluation.metrics import calculate_metrics, print_metrics
from src.visualization.plots import plot_forecast

%matplotlib inline

## 1. Prepare Data

In [ ]:
# Generate sample data
data = generate_sample_data(n_samples=365, trend=True, seasonality=True)

# Split into train and test
train, test = train_test_split_ts(data, test_size=0.2)

print(f"Training set size: {len(train)}")
print(f"Test set size: {len(test)}")

## 2. ARIMA Model

In [ ]:
# Create and train ARIMA model
arima_model = ARIMAForecaster(order=(2, 1, 2))
arima_model.fit(train)

# Make predictions
arima_predictions = arima_model.predict(steps=len(test))

# Evaluate
arima_metrics = calculate_metrics(test.values, arima_predictions.values)
print("ARIMA Model Performance:")
print_metrics(arima_metrics)

In [ ]:
# Visualize ARIMA forecast
plot_forecast(train, test, arima_predictions, title='ARIMA Forecast')

## 3. Prophet Model

In [ ]:
# Prepare data for Prophet
train_prophet = create_dataset_for_prophet(train)
test_prophet = create_dataset_for_prophet(test)

# Create and train Prophet model
prophet_model = ProphetForecaster(
    growth='linear',
    changepoint_prior_scale=0.05,
    seasonality_mode='additive'
)
prophet_model.fit(train_prophet)

# Make predictions
prophet_forecast = prophet_model.predict(periods=len(test), freq='D')
prophet_predictions = prophet_forecast['yhat'].tail(len(test)).values

# Evaluate
prophet_metrics = calculate_metrics(test.values, prophet_predictions)
print("Prophet Model Performance:")
print_metrics(prophet_metrics)

In [ ]:
# Visualize Prophet forecast
plot_forecast(train, test, prophet_predictions, title='Prophet Forecast')

## 4. Model Comparison

In [ ]:
# Compare models
comparison = pd.DataFrame({
    'ARIMA': arima_metrics,
    'Prophet': prophet_metrics
}).T

print("Model Comparison:")
print(comparison)

## Conclusion

This notebook demonstrated:
- ARIMA model training and forecasting
- Prophet model training and forecasting
- Model evaluation and comparison